In [1]:
#Python Libraries
from typing import List, Optional

#External
import numpy as np
import pandas as pd
from joblib import dump, load

#Sklearn
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.base import BaseEstimator, TransformerMixin


In [2]:
#Paths
EXEC_PATH="./assignment 3/execs_from_fix.csv"
QUOTES_PATH="./assignment 4/quotes_2025-09-10_small.csv.gz"

#experiment on these tickers first
KEEP_TICKERS = ["AAPL", "MSFT", "NVDA", "BRK.B", "HXHX", "SFHG", "DIDIY", "INEO", "THH", "FMCC", "LKNCY", "IVFH"]

MARKET_OPEN  = pd.to_datetime("09:30").time()
MARKET_CLOSE = pd.to_datetime("16:00").time()

In [3]:
def load_quotes(path: str, tickers: Optional[List[str]]) -> pd.DataFrame:
    dtypes = {
        "ticker":    "string",   # matches executions['Symbol']
        "bid_price": "float32",
        "ask_price": "float32",
        "bid_size":  "int32",
        "ask_size":  "int32",
        # bid_exchange / ask_exchange will default to object
    }

    chunk_size = 1_000_000
    chunks: List[pd.DataFrame] = []

    for chunk in pd.read_csv(
        path,
        compression="gzip",
        dtype=dtypes,
        low_memory=False,
        chunksize=chunk_size,
    ):
        if tickers is not None:
            chunk = chunk[chunk["ticker"].isin(tickers)]

        if chunk.empty:
            continue

        # ns since epoch -> datetime64[ns]
        chunk["sip_timestamp"] = pd.to_datetime(
            chunk["sip_timestamp"].astype("int64"),
            unit="ns",
            errors="coerce",
        )

        # keep only market hours
        t = chunk["sip_timestamp"].dt.time
        chunk = chunk[(t >= MARKET_OPEN) & (t <= MARKET_CLOSE)]

        if not chunk.empty:
            chunks.append(chunk)

    if chunks:
        quotes = pd.concat(chunks, ignore_index=True)
    else:
        quotes = pd.DataFrame(
            columns=[
                "ticker", "bid_price", "ask_price",
                "bid_size", "ask_size", "sip_timestamp",
                "bid_exchange", "ask_exchange",
            ]
        )

    return quotes

def load_executions(path: str, tickers: Optional[List[str]]) -> pd.DataFrame:
    dtypes = {
        "order_id": "string",
        "symbol": "string",
        "side": "Int8",          # 1 = buy, anything else = sell
        "order_qty": "Int32",
        "limit_price": "float32",
        "execution_price": "float32",
        "exchange": "string",
    }

    executions = pd.read_csv(
        path,
        dtype=dtypes,
        parse_dates=["order_time", "execution_time"],
        infer_datetime_format=True,
        low_memory=False,
    )

    if tickers is not None:
        executions = executions[executions["symbol"].isin(tickers)]

    # keep only market hours based on order time
    t = executions["execution_time"].dt.time
    executions = executions[(t >= MARKET_OPEN) & (t <= MARKET_CLOSE)]

    return executions

def attach_quotes(executions: pd.DataFrame, quotes: pd.DataFrame) -> pd.DataFrame:
    """
    Attach the most recent quote (per symbol) at or before OrderTransactTime.

    Assumes:
    - executions['Symbol'] and quotes['ticker'] are strings
    - executions['OrderTransactTime'] and quotes['sip_timestamp'] are datetime64[ns]
    """

    # merge_asof requires both sides sorted by the ON key (time), then BY key
    executions.sort_values(["execution_time", "symbol"], inplace=True)
    quotes.sort_values(["sip_timestamp", "ticker"], inplace=True)

    merged = pd.merge_asof(
        executions,
        quotes,
        left_on="execution_time",
        right_on="sip_timestamp",
        left_by="symbol",
        right_by="ticker",
        direction="backward",       # last quote <= order time
        allow_exact_matches=True,
    )

    return merged


In [4]:
quotes = load_quotes(QUOTES_PATH, tickers=None)
quotes

,ticker,bid_price,ask_price,bid_size,ask_size,bid_exchange,ask_exchange,sip_timestamp
0,A,125.360001,127.5,1,2,11,11,2025-09-10 11:50:02.929715281
1,A,124.349998,127.5,1,2,11,11,2025-09-10 11:59:01.349535066
2,A,123.750000,127.5,4,2,11,11,2025-09-10 11:59:01.350116469
3,A,125.389999,127.5,1,2,8,11,2025-09-10 11:59:01.350732905
4,A,124.379997,127.5,1,2,11,11,2025-09-10 12:01:52.220655733
...,...,...,...,...,...,...,...,...
173187377,ZZK,0.000000,0.0,0,0,3,3,2025-09-10 11:01:20.689921455
173187378,ZZK,25.000000,0.0,1,0,10,10,2025-09-10 11:01:23.363202324
173187379,ZZK,25.000000,0.0,1,0,11,0,2025-09-10 11:01:42.408600288
173187380,ZZK,0.000000,0.0,0,0,11,11,2025-09-10 11:01:46.689196820


In [5]:
executions = load_executions(EXEC_PATH, tickers=None)
executions

C:\Users\gonza\AppData\Local\Temp\ipykernel_9316\1859761274.py:65: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  executions = pd.read_csv(


,order_id,order_time,execution_time,symbol,side,order_qty,limit_price,execution_price,exchange
7844,ID86335,2025-09-10 09:30:00.997,2025-09-10 09:30:01.099,NIO,1,50,5.660000,5.660000,ID1516
7845,ID84205,2025-09-10 09:25:50.277,2025-09-10 09:30:01.498,AEHL,2,17,13.500000,13.500000,ID1516
7846,ID81598,2025-09-10 09:21:45.220,2025-09-10 09:30:02.167,UVIX,2,967,11.100000,11.100000,ID1516
7847,ID86355,2025-09-10 09:30:05.000,2025-09-10 09:30:05.103,MWYN,1,225,2.650000,2.650000,ID1516
7848,ID85769,2025-09-10 09:28:45.767,2025-09-10 09:30:06.735,HCAI,1,324,0.848200,0.848200,ID1516
...,...,...,...,...,...,...,...,...,...
41773,ID450310,2025-09-10 15:57:42.516,2025-09-10 15:59:58.806,TSLL,1,16488,13.800000,13.800000,ID1516
41774,ID446594,2025-09-10 15:53:29.975,2025-09-10 15:59:59.259,WLDS,2,26,6.800000,6.800000,ID1516
41775,ID451834,2025-09-10 15:59:35.993,2025-09-10 15:59:59.263,WLDS,2,942,6.800000,6.800000,ID1516
41776,ID452130,2025-09-10 15:59:57.656,2025-09-10 15:59:59.263,WLDS,2,11,6.800000,6.800000,ID1516


In [6]:
df_features = attach_quotes(executions, quotes)
df_features

,order_id,order_time,execution_time,symbol,side,order_qty,limit_price,execution_price,exchange,ticker,bid_price,ask_price,bid_size,ask_size,bid_exchange,ask_exchange,sip_timestamp
0,ID86335,2025-09-10 09:30:00.997,2025-09-10 09:30:01.099,NIO,1,50,5.660000,5.660000,ID1516,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaT
1,ID84205,2025-09-10 09:25:50.277,2025-09-10 09:30:01.498,AEHL,2,17,13.500000,13.500000,ID1516,AEHL,13.060000,13.380000,1.0,9.0,12.0,8.0,2025-09-10 09:30:00.891740082
2,ID81598,2025-09-10 09:21:45.220,2025-09-10 09:30:02.167,UVIX,2,967,11.100000,11.100000,ID1516,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaT
3,ID86355,2025-09-10 09:30:05.000,2025-09-10 09:30:05.103,MWYN,1,225,2.650000,2.650000,ID1516,MWYN,2.610000,2.650000,4.0,2.0,11.0,8.0,2025-09-10 09:30:05.102264552
4,ID85769,2025-09-10 09:28:45.767,2025-09-10 09:30:06.735,HCAI,1,324,0.848200,0.848200,ID1516,HCAI,0.848900,0.867400,4.0,1.0,11.0,12.0,2025-09-10 09:30:06.734577879
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33929,ID450310,2025-09-10 15:57:42.516,2025-09-10 15:59:58.806,TSLL,1,16488,13.800000,13.800000,ID1516,TSLL,13.800000,13.810000,174.0,809.0,12.0,11.0,2025-09-10 15:59:58.776039183
33930,ID446594,2025-09-10 15:53:29.975,2025-09-10 15:59:59.259,WLDS,2,26,6.800000,6.800000,ID1516,WLDS,6.740000,6.780000,1.0,1.0,12.0,7.0,2025-09-10 15:59:59.258771533
33931,ID451834,2025-09-10 15:59:35.993,2025-09-10 15:59:59.263,WLDS,2,942,6.800000,6.800000,ID1516,WLDS,6.770000,6.800000,1.0,52.0,12.0,8.0,2025-09-10 15:59:59.261720194
33932,ID452130,2025-09-10 15:59:57.656,2025-09-10 15:59:59.263,WLDS,2,11,6.800000,6.800000,ID1516,WLDS,6.770000,6.800000,1.0,52.0,12.0,8.0,2025-09-10 15:59:59.261720194


In [ ]:
RANDOM_SEED=42
RAW_FEATURE_COLS = [
    "side",        
    "order_qty",
    "limit_price",
    "execution_price",
    "bid_price",
    "ask_price",
    "bid_size",
    "ask_size",
    "bid_exchange",
    "ask_exchange",
]

TARGET_COL = "price_improvement"

def add_price_improvement(df: pd.DataFrame) -> None:
    """
    Adds 'price_improvement' to df in place.
    - Side 1 = buy → reference is ask_price
    - Side != 1 = sell → reference is bid_price
    Positive = better than NBBO.
    """
    is_buy = df["side"] == 1

    ref_price = np.where(is_buy, df["ask_price"], df["bid
        is_buy,
        ref_price - df["execution_price"],   # buy: cheaper than ask is good
        df["execution_price"] - ref_price,   # sell: higher than bid is good
    ).astype("float32")

    df["price_improvement"] = improvement

def prepare_data(df: pd.DataFrame) -> None:
    """Drop rows with missing raw features or target before training."""
    cols_needed = RAW_FEATURE_COLS + [TARGET_COL]
    df.dropna(subset=cols_needed, inplace=True)

In [8]:
df_features

,order_id,order_time,execution_time,symbol,side,order_qty,limit_price,execution_price,exchange,ticker,bid_price,ask_price,bid_size,ask_size,bid_exchange,ask_exchange,sip_timestamp
0,ID86335,2025-09-10 09:30:00.997,2025-09-10 09:30:01.099,NIO,1,50,5.660000,5.660000,ID1516,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaT
1,ID84205,2025-09-10 09:25:50.277,2025-09-10 09:30:01.498,AEHL,2,17,13.500000,13.500000,ID1516,AEHL,13.060000,13.380000,1.0,9.0,12.0,8.0,2025-09-10 09:30:00.891740082
2,ID81598,2025-09-10 09:21:45.220,2025-09-10 09:30:02.167,UVIX,2,967,11.100000,11.100000,ID1516,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaT
3,ID86355,2025-09-10 09:30:05.000,2025-09-10 09:30:05.103,MWYN,1,225,2.650000,2.650000,ID1516,MWYN,2.610000,2.650000,4.0,2.0,11.0,8.0,2025-09-10 09:30:05.102264552
4,ID85769,2025-09-10 09:28:45.767,2025-09-10 09:30:06.735,HCAI,1,324,0.848200,0.848200,ID1516,HCAI,0.848900,0.867400,4.0,1.0,11.0,12.0,2025-09-10 09:30:06.734577879
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33929,ID450310,2025-09-10 15:57:42.516,2025-09-10 15:59:58.806,TSLL,1,16488,13.800000,13.800000,ID1516,TSLL,13.800000,13.810000,174.0,809.0,12.0,11.0,2025-09-10 15:59:58.776039183
33930,ID446594,2025-09-10 15:53:29.975,2025-09-10 15:59:59.259,WLDS,2,26,6.800000,6.800000,ID1516,WLDS,6.740000,6.780000,1.0,1.0,12.0,7.0,2025-09-10 15:59:59.258771533
33931,ID451834,2025-09-10 15:59:35.993,2025-09-10 15:59:59.263,WLDS,2,942,6.800000,6.800000,ID1516,WLDS,6.770000,6.800000,1.0,52.0,12.0,8.0,2025-09-10 15:59:59.261720194
33932,ID452130,2025-09-10 15:59:57.656,2025-09-10 15:59:59.263,WLDS,2,11,6.800000,6.800000,ID1516,WLDS,6.770000,6.800000,1.0,52.0,12.0,8.0,2025-09-10 15:59:59.261720194


In [9]:
class FeatureBuilder(BaseEstimator, TransformerMixin):
    """Add engineered features for price-improvement models."""

    def fit(self, X: pd.DataFrame, y=None):
        return self
    
    def transform(self, X: pd.DataFrame) -> pd.DataFrame:
        # 1 = buy, anything else = sell
        side_num=np.where(X["side"] == 1, 1, -1).astype("int8")
        X["side_num"]=side_num

        #spread and mid
        X["spread"] = X["ask_price"] - X["bid_price"]
        X["mid_price"] = (X["ask_price"] + X["bid_price"]) / 2.0

        #distance between limit order and mid 
        X["limit_vs_mid"] = (X["limit_price"] - X["mid_price"]) * X["side_num"]

        cols = [
            "side_num",
            "order_qty",
            "limit_price",
            "bid_price",
            "ask_price",
            "bid_size",
            "ask_size",
            "bid_size",
            "bid_exchange",
            "ask_exchange",
            "spread",
            "mid_price",
            "limit_vs_mid",
        ]

        return X.loc[:, cols].astype("float32")

In [ ]:
def train_models_per_exchange(df: pd.DataFrame):
    """Train one regression model per exchange and report R² and MSE.

    Assumes `prepare_data(df)` has already been called so that:
    - side_num exists
    - rows with NaN in features/target have been dropped.
    """
    models: dict = {}
    metrics: dict = {}

    for exch, d in df.groupby("exchange"):
        # skip exchanges with too little data
        if len(d) < 50:
            continue

        X = d[RAW_FEATURE_COLS].astype("float32")
        y = d[TARGET_COL].astype("float32")

        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=RANDOM_SEED
        )

        pipe = Pipeline(
            steps=[
                ("features", FeatureBuilder()),
                ("scaler", StandardScaler()),
                ("model", RandomForestRegressor(random_state=RANDOM_SEED, n_jobs=-1)),
            ]
        )

        param_grid = {
            "model__n_estimators": [50, 100],
            "model__max_depth": [None, 10],
            "model__min_samples_leaf": [1,10]
        }

        grid = GridSearchCV(
            pipe,
            param_grid=param_grid,
            cv=3,
            n_jobs=-1,
            scoring="r2",
        )
        grid.fit(X_train, y_train)

        best_model = grid.best_estimator_

        # R2 on test set
        r2 = best_model.score(X_test, y_test)
        # MSE on test set
        y_pred = best_model.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)

        models[exch] = best_model
        metrics[exch] = {"r2": r2, "mse": mse}

        print(f"Exchange {exch}: R² = {r2:.3f}, MSE = {mse:.6f} (n={len(d)})")

    return models, metrics

In [11]:
add_price_improvement(df_features)
prepare_data(df_features)
models, metrics = train_models_per_exchange(df_features)
dump(models, "per_exchange_price_improvement_models.joblib")

Exchange ID1516: R² = 0.551, MSE = 0.045438 (n=33752)
Exchange ID29608: R² = 0.812, MSE = 0.002527 (n=60)


['per_exchange_price_improvement_models.joblib']

In [12]:
print("Number of exchanges with models:", len(models))
print("Exchanges:", list(models.keys()))

Number of exchanges with models: 2
Exchanges: ['ID1516', 'ID29608']


In [27]:
pipe = models[list(models.keys())[0]]
print(pipe)  # see the pipeline structure

rf = pipe.named_steps["model"]
print("n_estimators:", rf.n_estimators)
print("max_depth:", rf.max_depth)
print("min_samples_leaf:", rf.min_samples_leaf)

Pipeline(steps=[('features', FeatureBuilder()), ('scaler', StandardScaler()),
                ('model',
                 RandomForestRegressor(max_depth=10, n_estimators=50, n_jobs=-1,
                                       random_state=42))])
n_estimators: 50
max_depth: 10
min_samples_leaf: 1


In [14]:
print(metrics)

{'ID1516': {'r2': 0.5508349959680969, 'mse': 0.04543777661049099}, 'ID29608': {'r2': 0.8121056579226075, 'mse': 0.002527399885461546}}


In [15]:
len(df_features)

33877

In [26]:
from importlib import reload
import somewhat_smart_order_router as ssor

ssor = reload(ssor)

row = df_features.sample(1, random_state=0).iloc[0]
side_char = "B" if row["side"] == 1 else "S"

exch, expected_pi = ssor.best_price_improvement(
    symbol=row["symbol"],
    side=side_char,
    quantity=int(row["order_qty"]),
    limit_price=float(row["limit_price"]),
    bid_price=float(row["bid_price"]),
    ask_price=float(row["ask_price"]),
    bid_size=int(row["bid_size"]),
    ask_size=int(row["ask_size"]),
)

print("Best exchange:", exch)
print("Expected price improvement:", expected_pi)

KeyError: "['bid_exchange', 'ask_exchange'] not in index"